# SQL Server Connectivity through pyodbc

To Check for SQL Connectivity using pyodbc - let's look at how do we connect:
### 1. A compute Instance - run commands provided by link:

    https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15
### 2. A compute instance leveraging key vault

### 3. A compute cluster
    Create a docker image
    Leverate the docker image

# Compute instance - Import dependencies

**https://docs.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15**

**On your compute instnace run the following on the command prompt:**

```js
sudo su
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

#Download appropriate package for the OS version
#Choose only ONE of the following, corresponding to your OS version

#Ubuntu 16.04
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

#Ubuntu 18.04
curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

#Ubuntu 20.04
curl https://packages.microsoft.com/config/ubuntu/20.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

#Ubuntu 20.10
curl https://packages.microsoft.com/config/ubuntu/20.10/prod.list > /etc/apt/sources.list.d/mssql-release.list

exit
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17

#optional for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev
```


In [1]:
import azureml.core #adding core - this by default is in notebooks run on computer in Azure ML
from azureml.core import Workspace #needed for connecting to workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration, DEFAULT_GPU_IMAGE 
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.22.0 to work with mm-aml-dev


# Step 1: Get pyodbc to work on compute instance

In [3]:
import pyodbc 
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'servername.database.windows.net, 1433' 
database = 'databasename' 
username = 'user@servername' 
password = 'XXXX' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [4]:
#Sample select query
cursor.execute("select * from apiRequest") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchall()

1
(2, 1, 2, datetime.datetime(2020, 8, 10, 8, 0), True, 1, 'https://13-RRT-629.mktorest.com/bulk/v1/activities/export/{0}/enqueue.json', 60, -1)


# Work with secrets:
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-secrets-in-runs#:~:text=Set%20secrets%20In%20the%20Azure%20Machine%20Learning%2C%20the,the%20secret%20value%20if%20the%20name%20already%20exists.
    

In [5]:
server = os.environ.get("server")
print(server)
sqlServer = 'servername.database.windows.net, 1433' 
sqlDatabase = 'databasename'
sqlUsername = 'user@servername' 
sqlPassword = 'XXXX'

print(sqlServer)
print('********')
keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="sqlServer", value = sqlServer)
keyvault.set_secret(name="sqlDatabase", value = sqlDatabase)
keyvault.set_secret(name="sqlUsername", value = sqlUsername)
keyvault.set_secret(name="sqlPassword", value = sqlPassword)

secret_value = keyvault.get_secret(name="sqlServer")


None
mmm-adf-optimization.database.windows.net, 1433
********


# Setup secrets

In [6]:
import pyodbc

driver= '{ODBC Driver 17 for SQL Server}'

server   = keyvault.get_secret(name="sqlServer")
database = keyvault.get_secret(name="sqlDatabase")
username = keyvault.get_secret(name="sqlUsername")
password = keyvault.get_secret(name="sqlPassword")

connstr = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password
print(connstr)
with pyodbc.connect(connstr) as conn:

    
    with conn.cursor() as cursor:

        cursor.execute("select * from [dbo].[apiRequest]")

        row = cursor.fetchone()

        while row:

            print (str(row[0]) + " " + str(row[1]))

            row = cursor.fetchone()

DRIVER={ODBC Driver 17 for SQL Server};SERVER=mmm-adf-optimization.database.windows.net, 1433;PORT=1433;DATABASE=adf_config;UID=adfsa@mmm-adf-optimization;PWD=R0ckAndR0llAllNight
1 None
2 1


## Create Environment Based on image

https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-windows?tabs=azure-cli

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-custom-docker-image#create-a-custom-base-image

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-custom-docker-image#use-a-custom-base-image

**Docker File below**

```js
FROM mcr.microsoft.com/azureml/o16n-sample-user-base/ubuntu-miniconda:latest


# apt-get and system utilities
RUN apt-get update && apt-get install -y \
    curl apt-utils apt-transport-https debconf-utils gcc build-essential g++-5\
    && rm -rf /var/lib/apt/lists/*

# adding custom MS repository
RUN curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
RUN curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list

# install SQL Server drivers
RUN apt-get update && ACCEPT_EULA=Y apt-get install -y msodbcsql unixodbc-dev

# install SQL Server tools
RUN apt-get update && ACCEPT_EULA=Y apt-get install -y mssql-tools
RUN echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
RUN /bin/bash -c "source ~/.bashrc"
```


In [7]:
# Create the environment
myenv = Environment(name="myenv")
# Enable Docker and reference an image
myenv.docker.enabled = True
myenv.docker.base_image = "7b68fa5b8a1f4f8abe3d54389e9b015c.azurecr.io/pyodbcimage:latest"

# Define the packages needed by the model and scripts
from azureml.core.conda_dependencies import CondaDependencies
conda_dep = CondaDependencies()
# you must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("pyodbc")
conda_dep.add_pip_package("numpy==1.19.4")
myenv.python.conda_dependencies=conda_dep



## Create Cluster for Training

In [8]:
cluster_name = 'mmdsvm04x'
try:
    compute_target = ComputeTarget(workspace=ws,  name=cluster_name )
    print('found existing:', compute_target.name)
    
except:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_DS11_V2',
        min_nodes=0,
        max_nodes=1)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

found existing: mmdsvm04x


In [9]:
cwd = os.getcwd()
current_dir = cwd
print(cwd)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mmcompute/code/Users/memasanz/Marathon


In [10]:
import os
script_folder = os.path.join(os.getcwd(), "train")
print(script_folder)
os.makedirs(script_folder, exist_ok = True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mmcompute/code/Users/memasanz/Marathon/train


In [32]:
%%writefile $script_folder/train.py
import pyodbc
import sys
import os
import numpy as np
from azureml.core import Run
import random

# DEFINE THE MODEL
def connect_sql():

    print('get connect')
    run = Run.get_context()

    driver= '{ODBC Driver 17 for SQL Server}'
    
    server   = run.get_secret(name="sqlServer")
    database = run.get_secret(name="sqlDatabase")
    username = run.get_secret(name="sqlUsername")
    password = run.get_secret(name="sqlPassword")
                              
    run.log("server", server)

    connstr = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password
    print(connstr)
    with pyodbc.connect(connstr) as conn:
        with conn.cursor() as cursor:
            cursor.execute("select * from [dbo].[apiRequest]")
            row = cursor.fetchone()
            while row:
                print (str(row[0]) + " " + str(row[1]))
                row = cursor.fetchone()


def main():
    connect_sql()
    
if __name__ == "__main__":
    main()

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/mmcompute/code/Users/memasanz/Marathon/train/train.py


In [33]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.widgets import RunDetails

experiment = Experiment(workspace = ws, name = "exp-connect-sql")
script_config = ScriptRunConfig(source_directory = script_folder, script = 'train.py', environment=myenv, compute_target = cluster_name)

experiment = Experiment(workspace=ws, name = "exp-connect-sql" )
run = experiment.submit(config= script_config)

In [34]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…